## Embedding model ##
>Floris Menninga \
>Datum: 8-01-2026

In [ ]:
# %pip install tensorflow keras
# %pip install tqdm
# %pip install tensorflow_text
#%pip install bs4
%pip install joblib

from data_parser import xml_parser

import io
import re
import string
import tqdm

import numpy as np
import requests
import tensorflow as tf
import tensorflow_text as tf_text
import joblib
import tensorflow as tf
from tensorflow.keras import layers

%load_ext tensorboard
AUTOTUNE = tf.data.AUTOTUNE
SEED = 0

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
#Gevonden op het internet, om de GPU versnelling te laten werken:
%export LD_LIBRARY_PATH=$(find .venv -name "*.so*" | grep nvidia | xargs dirname | sort -u | paste -d ":" -s -)

## Tokenizer

De eerste stap is het omzetten van de text in tokens:


In [4]:
sentence = "Hallo, dit is een testzin om het programma te testen."
tokens = list(sentence.lower().split())
print(len(tokens))

10


### Vocabulary maken van deze tokens:

In [5]:
vocab, index = {}, 1 
vocab['<pad>'] = 0
for token in tokens:
  if token not in vocab:
    vocab[token] = index
    index += 1
vocab_size = len(vocab)

print(vocab)

{'<pad>': 0, 'hallo,': 1, 'dit': 2, 'is': 3, 'een': 4, 'testzin': 5, 'om': 6, 'het': 7, 'programma': 8, 'te': 9, 'testen.': 10}


### Inverse vocabulary:
Van integer index naar token:

In [6]:
inverse_vocab = {index: token for token, index in vocab.items()}

print(inverse_vocab)

{0: '<pad>', 1: 'hallo,', 2: 'dit', 3: 'is', 4: 'een', 5: 'testzin', 6: 'om', 7: 'het', 8: 'programma', 9: 'te', 10: 'testen.'}


## Vectorizeren van de zin:

In [7]:
test_sequence = [vocab[word] for word in tokens]

print(test_sequence)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


## Skip-gram maken:


In [8]:
window_size = 2

positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
      test_sequence,
      vocabulary_size=vocab_size,
      window_size=window_size,
      negative_samples=0,
      seed=0
)

print(len(positive_skip_grams))

34


In [9]:
for target, context in positive_skip_grams[:5]:
  print(f"({target}, {context}): ({inverse_vocab[target]}, {inverse_vocab[context]})")


(3, 4): (is, een)
(1, 2): (hallo,, dit)
(8, 9): (programma, te)
(6, 8): (om, programma)
(4, 3): (een, is)


In [ ]:
target_word, context_word = positive_skip_grams[0]

num_ns = 10

context_class = tf.reshape(tf.constant(context_word, dtype="int64"), (1, 1))
negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
    true_classes=context_class,  # class that should be sampled as 'positive'
    num_true=1,  # each positive skip-gram has 1 positive context class
    num_sampled=num_ns,  # number of negative context words to sample
    unique=True,  # all the negative samples should be unique
    range_max=vocab_size,  # pick index of the samples from [0, vocab_size]
    seed=SEED,  # seed for reproducibility
    name="negative_sampling"  # name of this operation
)
print(negative_sampling_candidates)
print([inverse_vocab[index.numpy()] for index in negative_sampling_candidates])


tf.Tensor([6 1 5 0], shape=(4,), dtype=int64)
['om', 'hallo,', 'testzin', '<pad>']


W0000 00:00:1767992958.724316 1150736 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [11]:
# Reduce a dimension so you can use concatenation (in the next step).
squeezed_context_class = tf.squeeze(context_class, 1)

# Concatenate a positive context word with negative sampled words.
context = tf.concat([squeezed_context_class, negative_sampling_candidates], 0)

# Label the first context word as `1` (positive) followed by `num_ns` `0`s (negative).
label = tf.constant([1] + [0]*num_ns, dtype="int64")
target = target_word


In [12]:
print(f"target_index    : {target}")
print(f"target_word     : {inverse_vocab[target_word]}")
print(f"context_indices : {context}")
print(f"context_words   : {[inverse_vocab[c.numpy()] for c in context]}")
print(f"label           : {label}")


target_index    : 3
target_word     : is
context_indices : [4 6 1 5 0]
context_words   : ['een', 'om', 'hallo,', 'testzin', '<pad>']
label           : [1 0 0 0 0]


In [13]:
sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(size=10)
print(sampling_table)


[0.00315225 0.00315225 0.00547597 0.00741556 0.00912817 0.01068435
 0.01212381 0.01347162 0.01474487 0.0159558 ]


In [ ]:
# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  # Elements of each training example are appended to these lists.
  targets, contexts, labels = [], [], []

  # Build the sampling table for `vocab_size` tokens.
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in the dataset.
  for sequence in tqdm.tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence,
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=10,
          min_count = 5,
          seed=0)

    # Iterate over each positive skip-gram pair to produce training examples
    # with a positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=num_ns,
          unique=True,
          range_max=vocab_size,
          seed=0,
          name="negative_sampling")

      # Build context and label vectors (for one target word)
      context = tf.concat([tf.squeeze(context_class,1), negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      # Append each element from the training example to global lists.
      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels


## Word2Vec:


In [18]:
text_ds = tf.data.TextLineDataset(["/run/media/floris/FLORIS_3/Data_set/PubMed/dataset_100914.txt"]).filter(lambda x: tf.cast(tf.strings.length(x), bool))


In [19]:
# Now, create a custom standardization function to lowercase the text and
# remove punctuation.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation), '')


# Define the vocabulary size and the number of words in a sequence.
sequence_length = 128 
vocab_size = 50000

# Use the `TextVectorization` layer to normalize, split, and map strings to
# integers. Set the `output_sequence_length` length to pad all samples to the
# same length.
vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)


In [20]:
vectorize_layer.adapt(text_ds.batch(512))


2026-01-09 22:59:53.585434: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2026-01-09 22:59:53.588822: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 147713328 exceeds 10% of free system memory.
2026-01-09 23:00:05.680337: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 147713328 exceeds 10% of free system memory.


In [21]:
# Save the created vocabulary for reference.
inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:20])


['', '[UNK]', np.str_('the'), np.str_('of'), np.str_('and'), np.str_('in'), np.str_('to'), np.str_('a'), np.str_('with'), np.str_('for'), np.str_('were'), np.str_('was'), np.str_('that'), np.str_('cells'), np.str_('is'), np.str_('by'), np.str_('as'), np.str_('cancer'), np.str_('or'), np.str_('from')]


In [22]:
# Vectorize the data in text_ds.
text_vector_ds = text_ds.batch(512).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()


In [23]:
sequences = list(text_vector_ds.as_numpy_iterator())
print(len(sequences))


536766


In [24]:
# for seq in sequences[:5]:
#   print(f"{seq} => {[inverse_vocab[i] for i in seq]}")


In [ ]:
targets, contexts, labels = generate_training_data(
    sequences=sequences,
    window_size=5,
    num_ns=10,
    vocab_size=vocab_size,
    seed=0)

targets = np.array(targets)
contexts = np.array(contexts)
labels = np.array(labels)

print('\n')
print(f"targets.shape: {targets.shape}")
print(f"contexts.shape: {contexts.shape}")
print(f"labels.shape: {labels.shape}")

  0%|          | 0/536766 [00:00<?, ?it/s]

 14%|█▍        | 74034/536766 [23:59<2:01:34, 63.44it/s] 

Gegenereerde data opslaan:
Het duurde lang om te genereren dus sla ik het op in een .joblib bestand, dit zal ik ook voor sommige vervolg stappen doen.


In [ ]:
joblib_list = [targets, contexts, labels]

joblib.dump(joblib_list, "targets_contexts_labels.joblib")

In [ ]:
BATCH_SIZE = 20
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset)

<_PrefetchDataset element_spec=((TensorSpec(shape=(20,), dtype=tf.int64, name=None), TensorSpec(shape=(20, 5), dtype=tf.int64, name=None)), TensorSpec(shape=(20, 5), dtype=tf.int64, name=None))>


In [ ]:
class Word2Vec(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = layers.Embedding(vocab_size,
                                      embedding_dim,
                                      name="w2v_embedding")
    self.context_embedding = layers.Embedding(vocab_size,
                                       embedding_dim)

  def call(self, pair):
    target, context = pair
    # target: (batch, dummy?)  # The dummy axis doesn't exist in TF2.7+
    # context: (batch, context)
    if len(target.shape) == 2:
      target = tf.squeeze(target, axis=1)
    # target: (batch,)
    word_emb = self.target_embedding(target)
    # word_emb: (batch, embed)
    context_emb = self.context_embedding(context)
    # context_emb: (batch, context, embed)
    dots = tf.einsum('be,bce->bc', word_emb, context_emb)
    # dots: (batch, context)
    return dots

In [ ]:
def custom_loss(x_logit, y_true):
      return tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=y_true)

In [ ]:
embedding_dim = 256
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [ ]:
word2vec.fit(dataset, epochs=10, callbacks=[tensorboard_callback])

Epoch 1/50
515/515 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9844 - loss: 0.1236
Epoch 2/50
515/515 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9853 - loss: 0.1056
Epoch 3/50
515/515 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9861 - loss: 0.0924
Epoch 4/50
515/515 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9866 - loss: 0.0827
Epoch 5/50
515/515 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9866 - loss: 0.0753
Epoch 6/50
515/515 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9868 - loss: 0.0698
Epoch 7/50
515/515 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9868 - loss: 0.0655
Epoch 8/50
515/515 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9868 - loss: 0.0621
Epoch 9/50
515/515 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9868 - loss: 0.0596
Epoch 10/50
515/515 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9867 - loss: 0.0575
Epoch 11/50
515/515 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9868 - loss: 0.0559
Epoch 12/50
515/515 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step

In [ ]:
weights = word2vec.get_layer('w2v_embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [ ]:
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [ ]:
# try:
#   from google.colab import files
#   files.download('vectors.tsv')
#   files.download('metadata.tsv')
# except Exception:
#   pass


## Data:
Als trainingsdata gebruik ik 869597 Pubmed artikelen (~150GB) die ik gedownload heb van https://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_bulk/oa_comm/xml/

oa_comm_xml.PMC000xxxxxx.baseline.2025-06-26.tar.gz t/m oa_comm_xml.PMC004xxxxxx.baseline.2025-06-26.tar.gz

Deze zijn nieuwer dan de artikelen uit 2021 die op /commons/data/NCBI/PubMed/ staan.

Vervolgens moeten deze artikelen doorzocht worden op inhoud. \
De .xml bestanden hebben headers voor titel, abstract, body etc. deze doorzoek ik met behulp van mijn classe: data_parser.py

Het filteren op de drie keywords "cancer", "melanoma" en "carcinoma" duurde 9 uur en 30 minuten en reduceerde de hoeveelheid artikelen 
van 869597 naar 100914 (2.7GB).

100914 Artikelen met deze zoekcriteria opgeslagen in: /run/media/floris/FLORIS_3/Data_set/PubMed/dataset_large.txt

In [ ]:
keyword_list = ["cancer", "melanoma", "carcinoma"] # Niet hoofdlettergevoelig...

trainings_data = xml_parser("/run/media/floris/73ab731c-036f-4277-83dc-b38482cf146b/Data_Sets/Pubmed/oa_comm_xml.PMC002xxxxxx.baseline.2025-06-26/PMC002xxxxxx/", "/run/media/floris/FLORIS_2/data/trainingsData.txt", keyword_list)

trainings_data.run()



122576 XML bestanden...


 38%|███▊      | 47156/122576 [25:50<52:03, 24.15it/s]  

Error /run/media/floris/73ab731c-036f-4277-83dc-b38482cf146b/Data_Sets/Pubmed/oa_comm_xml.PMC002xxxxxx.baseline.2025-06-26/PMC002xxxxxx/: 'NoneType' object has no attribute 'get_text'


 51%|█████▏    | 62919/122576 [36:46<20:15, 49.06it/s]  

Error /run/media/floris/73ab731c-036f-4277-83dc-b38482cf146b/Data_Sets/Pubmed/oa_comm_xml.PMC002xxxxxx.baseline.2025-06-26/PMC002xxxxxx/: 'NoneType' object has no attribute 'get_text'


 53%|█████▎    | 65450/122576 [38:17<33:25, 28.49it/s]  


KeyboardInterrupt: 